# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


# Carregando o dataset
df = pd.read_csv('previsao_de_renda.csv')

# Convertendo a coluna 'data_ref' para datetime
df['data_ref'] = pd.to_datetime(df['data_ref'])

# Análise exploratória inicial
print(df.info())
print(df.describe())

In [ ]:
# Verificando valores nulos
print(df.isnull().sum())

In [ ]:
# Tratando valores nulos em 'tempo_emprego' (substituindo por mediana)
median_tempo_emprego = df['tempo_emprego'].median()
df['tempo_emprego'] = df['tempo_emprego'].fillna(median_tempo_emprego)

# Conferindo valores nulos
print(df.isnull().sum())

In [ ]:
# Verificando outliers em 'idade' e 'renda' (usando boxplot)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x=df['idade'])
plt.subplot(1, 2, 2)
sns.boxplot(x=df['renda'])
plt.show()

In [ ]:
# Calculando o IQR para a coluna 'renda'
Q1 = df['renda'].quantile(0.25)
Q3 = df['renda'].quantile(0.75)
IQR = Q3 - Q1

# Definindo os limites
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Removendo os outliers da coluna 'idade'
df = df[(df['renda'] >= lower_bound) & (df['renda'] <= upper_bound)]


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 2)
sns.boxplot(x=df['renda'])
plt.show()

In [ ]:
# Verificando valores únicos em variáveis categóricas
for col in df.select_dtypes(include='object'):
    print(col, df[col].unique())


In [ ]:
# Codificando variáveis categóricas (exemplo: one-hot encoding)
df = pd.get_dummies(df, columns=['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'])

# Criando novas features (exemplo: faixa etária)
df['faixa_etaria'] = pd.cut(df['idade'], bins=[18, 30, 45, 60, 100], labels=['18-30', '31-45', '46-60', '60+'])

# Salvando o dataframe limpo
df.to_csv('previsao_de_renda_limpo.csv', index=False)

#### Passo 1 - Separe a base em treinamento e teste (25% para teste, 75% para treinamento) ####

In [96]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carregando o dataset limpo
df = pd.read_csv('previsao_de_renda_limpo.csv')

# Separando as features (X) e a variável target (y)
X = df.drop('renda', axis=1)
y = df['renda']

# Dividindo os dados em treinamento e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [98]:
print(X_train.columns)

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'sexo_F', 'sexo_M', 'tipo_renda_Assalariado',
       'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor público',
       'educacao_Primário', 'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Casado', 'estado_civil_Separado', 'estado_civil_Solteiro',
       'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Aluguel',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental', 'faixa_etaria'],
      dtype='object')


#### Passo 2 - Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o na base de testes. Qual o melhor modelo? ####

In [100]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# Carregando o dataset limpo
data = pd.read_csv('previsao_de_renda_limpo.csv')

# Removendo a coluna irrelevante
data.drop('Unnamed: 0', axis=1, inplace=True)

# Convertendo a coluna 'data_ref' para datetime e extraindo features
data['data_ref'] = pd.to_datetime(data['data_ref'])
data['ano'] = data['data_ref'].dt.year
data['mes'] = data['data_ref'].dt.month
data['dia'] = data['data_ref'].dt.day
data.drop('data_ref', axis=1, inplace=True)

# Separando as features (X) e a variável target (y)
# (Substitua 'target' pelo nome da sua variável alvo)
X = data.drop('renda', axis=1)
y = data['renda']

# Identificando as colunas categóricas
categorical_cols = X.select_dtypes(include=['object']).columns

# Criando um codificador OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Ajustando o codificador aos dados de treino
encoder.fit(X[categorical_cols])

# Transformando os dados de treino e teste
X_encoded = pd.DataFrame(encoder.transform(X[categorical_cols]), columns=encoder.get_feature_names_out())
X = pd.concat([X.drop(categorical_cols, axis=1), X_encoded], axis=1)

# Dividindo os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Lista de valores de alpha
alpha_values = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Dicionário para armazenar os resultados
resultados = {}

# Loop para testar diferentes valores de alpha
for alpha in alpha_values:
    # Criando o modelo
    model = Ridge(alpha=alpha)
    
    # Treinando o modelo
    model.fit(X_train, y_train)
    
    # Fazendo previsões
    y_pred = model.predict(X_test)
    
    # Calculando o R²
    r2 = r2_score(y_test, y_pred)
    resultados[alpha] = r2

# Imprimindo os resultados
for alpha, r2 in resultados.items():
    print(f"Alpha: {alpha}, R²: {r2}")

# Encontrando o melhor modelo (o que tem o maior R²)
melhor_alpha = max(resultados, key=resultados.get)
print(f"O melhor valor de alpha é {melhor_alpha} com R² de {resultados[melhor_alpha]}")

Alpha: 0, R²: 0.23727809339425499
Alpha: 0.001, R²: 0.23728806758854104
Alpha: 0.005, R²: 0.23728820394600125
Alpha: 0.01, R²: 0.2372883742671016
Alpha: 0.05, R²: 0.2372897318327194
Alpha: 0.1, R²: 0.2372914164052492
O melhor valor de alpha é 0.1 com R² de 0.2372914164052492


#### Passo 3 - Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado? ####

In [102]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder

# Dicionário para armazenar os resultados
resultados = {}
# Loop para testar diferentes valores de alpha
for alpha in alpha_values:
    if alpha == 0:
        model = LinearRegression()
    else:
        model = Lasso(alpha=alpha, max_iter=10000)  # Aumentando o número máximo de iterações
    
    # Treinando o modelo
    model.fit(X_train, y_train)
    
    # Fazendo previsões
    y_pred = model.predict(X_test)
    
    # Calculando o R²
    r2 = r2_score(y_test, y_pred)
    resultados[alpha] = r2

# Imprimindo os resultados
for alpha, r2 in resultados.items():
    print(f"Alpha: {alpha}, R²: {r2}")

# Encontrando o melhor modelo (o que tem o maior R²)
melhor_alpha = max(resultados, key=resultados.get)
print(f"O melhor valor de alpha é {melhor_alpha} com R² de {resultados[melhor_alpha]}")

Alpha: 0, R²: 0.23728803348519056
Alpha: 0.001, R²: 0.23728945401032675
Alpha: 0.005, R²: 0.23729327914155296
Alpha: 0.01, R²: 0.23729849953703097
Alpha: 0.05, R²: 0.2373393544280642
Alpha: 0.1, R²: 0.23738815013223413
O melhor valor de alpha é 0.1 com R² de 0.23738815013223413


Comparando somente o R² dos métodos(Ridge e Lasso) o maior resultado é o alpha de Lasso: 0.23738815013223413

#### Passo 4 - Rode um modelo stepwise. Avalie o na base de testes. Qual o melhor resultado? ####

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import pandas as pd

# Carregar os dados
data = pd.read_csv("previsao_de_renda_limpo.csv")

# Converter a coluna 'data_ref' para datetime
data['data_ref'] = pd.to_datetime(data['data_ref'])

# Criar features numéricas a partir da data 
data['ano'] = data['data_ref'].dt.year
data['mes'] = data['data_ref'].dt.month
data['dia_semana'] = data['data_ref'].dt.dayofweek

# Remover a coluna original de data 
data.drop('data_ref', axis=1, inplace=True)

# Tratar a coluna 'faixa_etaria' (exemplo usando one-hot encoding)
data = pd.get_dummies(data, columns=['faixa_etaria'])


# Separar as features (X) e a variável target (y)
X = data.drop('renda', axis=1)
y = data['renda']

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar um modelo de regressão linear
model = LinearRegression()

# Selecionar as melhores features usando o método stepwise forward
sfs = SequentialFeatureSelector(model, direction='forward', scoring='r2')
sfs.fit(X_train, y_train)

# Obter as features selecionadas
selected_features = X_train.columns[sfs.get_support()]

# Treinar o modelo final com as features selecionadas
final_model = LinearRegression()
final_model.fit(X_train[selected_features], y_train)

# Fazer previsões na base de testes
y_pred = final_model.predict(X_test[selected_features])

# Calcular o R² na base de testes
r2 = r2_score(y_test, y_pred)
print("R² na base de testes:", r2)

R² na base de testes: 0.24502957887321497


#### Passo 5 - Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos? ####

Ridge: Esse modelo é como um treinador que tenta equilibrar o time, evitando que um jogador se destaque demais e desbalanceie o jogo. Ele evita que o modelo fique "super treinado" (overfitting), mas às vezes não consegue identificar os jogadores mais importantes do time (features mais relevantes).

Lasso: O Lasso é um treinador mais rigoroso. Ele não só equilibra o time, mas também corta os jogadores que não contribuem muito para o resultado final. Por isso, ele é bom em selecionar os jogadores-chave (features importantes).

Stepwise: O Stepwise é como um técnico que vai montando o time aos poucos, testando diferentes combinações até encontrar a melhor formação. Ele é bem eficiente em encontrar a melhor equipe, mas às vezes pode escolher jogadores por acaso, sem um critério muito claro.

Quem venceu o jogo?

No nosso caso, o Stepwise levou a melhor. Ele conseguiu montar a melhor equipe (modelo) e obteve o melhor resultado. Isso mostra que a estratégia de selecionar as features de forma gradual foi a mais eficiente para este conjunto de dados.

#### Passo 6 - Partindo dos modelos que você ajustou, tente melhorar o R² na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis. ####

In [118]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# ... (seu código existente)

# Engenharia de features
data['experiencia_profissional_por_idade'] = data['tempo_emprego'] / data['idade']

# Seleção de features
selector = SelectFromModel(RandomForestRegressor(n_estimators=100))
selector.fit(X_train, y_train)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Pipeline com pré-processamento e modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor())
])

# Grid search para ajustar os hiperparâmetros
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 5, 10]
}
# ... (utilizar GridSearchCV para encontrar os melhores parâmetros)

# Treinar o modelo final
pipeline.fit(X_train_selected, y_train)

# Avaliar o modelo na base de testes
y_pred = pipeline.predict(X_test_selected)

# Calcular o erro quadrático médio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Erro quadrático médio (MSE):", mse)

# Calcular o coeficiente de determinação (R²)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinação (R²):", r2) 

Erro quadrático médio (MSE): 5046560.735102035
Coeficiente de determinação (R²): 0.33919752775969436


#### Passo 7 - Ajuste uma árvore de regressão e veja se consegue um melhor com ela. ####

In [140]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

# Criando um pipeline com uma árvore de decisão
tree_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', DecisionTreeRegressor())
])

# Ajustando os hiperparâmetros da árvore de decisão
param_grid = {
    'model__max_depth': [3, 5, 7, None],  # Incluindo None para permitir que a árvore cresça ao máximo
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__ccp_alpha': [0.0, 0.01, 0.05]  # Complexidade de poda de custo
}

# Realizando a busca em grade
grid_search = GridSearchCV(tree_pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train_selected, y_train)

# Obtendo o melhor modelo e seus parâmetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Seleção de features (Exemplo usando SelectFromModel)
selector = SelectFromModel(RandomForestRegressor(n_estimators=100))
selector.fit(X_train, y_train)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test) 


# Balanceamento de classes (Exemplo usando SMOTE)
if np.unique(y_train).size == 2:  # Verificar se é um problema de classificação binária
    smote = SMOTE()
    X_train_res, y_train_res = smote.fit_resample(X_train_selected, y_train)
    # Utilizar X_train_res e y_train_res para treinar o modelo

# Fazendo previsões com o melhor modelo
y_pred = best_model.predict(X_test_selected)

# Calculando as métricas para a árvore de decisão
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Melhores parâmetros:", best_params)
print("MSE da árvore de decisão:", mse)
print("R² da árvore de decisão:", r2)

Melhores parâmetros: {'model__ccp_alpha': 0.01, 'model__max_depth': 3, 'model__min_samples_leaf': 1, 'model__min_samples_split': 5}
MSE da árvore de decisão: 5808550.67248494
R² da árvore de decisão: 0.23942168815817655
